In [1]:
from ngsolve import *
from netgen.occ import *
from netgen.geom2d import SplineGeometry
from ngsolve.internal import *
from xfem import *
import numpy as np
import ipywidgets
from ngsolve.solvers import *
import gc
def MakeGeometry():
    geo = SplineGeometry()

    # Define Points:|
    p0 = geo.AppendPoint(0.0,     0.0)
    p0_inset = geo.AppendPoint(0.00005,0.0)
    p1 = geo.AppendPoint(0.001,   0.0)
    p1b = geo.AppendPoint(0.002,  0.0)
    p2 = geo.AppendPoint(0.007, 0.0)
    p3_inset = geo.AppendPoint(0.00995,0.0)
    p3 = geo.AppendPoint(0.01, 0.0)
    
    p4 = geo.AppendPoint(0.01, 0.10)
    p5 = geo.AppendPoint(0.008, 0.10)
    p1b_top = geo.AppendPoint(0.002,  0.10)
    
    p6 = geo.AppendPoint(0.001,   0.10)
    p7 = geo.AppendPoint(0.00,   0.10)
    p7_inset = geo.AppendPoint(0.00005,0.10)

    # Bottom edges
 
    geo.Append(["line", p0, p1],     leftdomain=1, rightdomain=0, bc="Liq_Outlet_a")
    geo.Append(["line", p1, p1b],    leftdomain=2, rightdomain=0, bc="Liq_Outlet_b")  # new domain
    geo.Append(["line", p1b, p2],    leftdomain=3, rightdomain=0, bc="bottom")
    geo.Append(["line", p2, p3_inset],     leftdomain=4, rightdomain=0, bc="Gas_Inlet")
    geo.Append(["line",p3_inset,p3], leftdomain=4 ,rightdomain=0, bc = "Gas_Buffer")
    
    # Right edge
    geo.Append(["line", p3, p4],     leftdomain=4, rightdomain=0, bc="right")

    # Top edges
    geo.Append(["line", p4, p5],     leftdomain=4, rightdomain=0, bc="Gas_Outlet")
    geo.Append(["line", p5, p1b_top],leftdomain=3, rightdomain=0, bc="top")
    geo.Append(["line", p1b_top, p6],leftdomain=2, rightdomain=0, bc="top")     # new domain
    geo.Append(["line", p6, p7_inset],     leftdomain=1, rightdomain=0, bc="Liq_Inlet")
    geo.Append(["line", p7_inset,p7],     leftdomain=1, rightdomain=0, bc="Liq_Buffer")

    # Left edge
    geo.Append(["line", p7, p0],     leftdomain=1, rightdomain=0, bc="left")

    # Internal vertical separators
    interface = geo.Append(["line", p1, p6],       leftdomain=1, rightdomain=2)  # 
    geo.Append(["line", p1b, p1b_top], leftdomain=2, rightdomain=3)  # 
    geo.Append(["line", p2, p5],       leftdomain=3, rightdomain=4)  #



    # Set mesh size per domain

    geo.SetDomainMaxH(1, 0.0001)
    geo.SetDomainMaxH(2, 0.0001)  # new domain (optional: adjust mesh size)
    geo.SetDomainMaxH(3, 0.0007)
    geo.SetDomainMaxH(4, 0.0005)

    return geo.GenerateMesh()


mesh =  Mesh(MakeGeometry())

importing ngsxfem-2.1.2405


**Imports and Function Spaces**

In [2]:
mesh = Mesh("mesh.vol")


In [3]:
# =============================================================================
# FINITE ELEMENT SPACES
# =============================================================================

# Navier-Stokes spaces
V = VectorH1(mesh, order=2, dirichlet="Liq_Inlet|Gas_Inlet|top|bottom|left|right|Liq_Buffer|Gas_Buffer") #dirichlety = "top|bottom"
Q = H1(mesh, order=1,dirichlet="")  # pressure constraints
X = V*Q  # Mixed velocity-pressure space

# AllenCahn Space
L = H1(mesh, order=1,dirichlet="left|Liq_Inlet|Gas_Inlet|Gas_Outlet|Gas_Buffer|top") 

# GRID FUNCTIONS
# =============================================================================

# Navier-Stokes solutions
u_p = GridFunction(X)      # Current [u, p] solution
u_p_old = GridFunction(X)  # Previous time step [u, p]

# Phase field solutions  
phi = GridFunction(L)      # Current [phi, mu] solution
phi_old = GridFunction(L)  # Previous time step [phi, mu]
# Boundary condition helpers|
u_D = GridFunction(V)  # Velocity BC helper
p_D = GridFunction(Q)  # Pressure BC helper

# Concentration space (continuous Galerkin)
C = H1(mesh, order=1, dirichlet="")  # set your true inflow boundaries
c_i     = GridFunction(C)   # current c^{n+1}
c_i_old = GridFunction(C)   # previous c^n


# TEMP
velocity_pressure = np.load("Data/U_snap_freq24.000Hz.npy")

u_snapshots = velocity_pressure[:,:V.ndof]
phi_snapshots = np.load("Data/Phi_snap_freq24.000Hz.npy")





FileNotFoundError: [Errno 2] No such file or directory: 'Data/U_snap_freq24.000Hz.npy'

In [4]:

phi.vec.FV().NumPy()[:] = (phi_snapshots[0,:])
phi_old.Set(phi)

# Initialize Navier-Stokes (can be zero or small perturbation)
u_p.components[0].vec.FV().NumPy()[:] = u_snapshots[0]
u_p.components[1].Set(0.0)             # Initial pressure
u_p_old.vec.data = u_p.vec

In [5]:
Draw(u_p.components[0],mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

**Physical System Parameters**

In [6]:
# ---------------- 3) Physical parameters ----------------
# Molecular diffusivities
Dil = Parameter(2.0e-9)                        # m^2/s (CO2 in water) (SLOWER)
Dig = Parameter(1.5e-5)                        # m^2/s (CO2 in air)



# Reaction parameters (CO2–MEA)
T      = Parameter(298.15)                      # Temp in K
phiMEA = Parameter(0.30)                        # MEA mole fraction
thetaL = Parameter(0.10)                        # CO2 loading
KCO2   = Parameter(1.0e-5)                      # equilibrium constant (THIS NEEDS TO BE CHANGED)
Rgas   = 8.314462618                            # J/(mol K)
cMEA   = Parameter(1000.0)                      # mol/m^3 (MEA concentration)
k_i = Parameter(1.20) #Dimensionless Henry, For now. 1.20. Check later


CO2_Loading = 0.3 #theta

# ---------------- Quick conversions ----------------
M_MEA   = 61.08     # g/mol
M_H2O   = 18.015    # g/mol

rho_sol = 1050.0       # kg/m^3  (approx for 25 wt% MEA)
M_MEA_Kg   = 0.06108      # kg/mol
w_MEA   = 0.25         # MEA mass fraction

C_MEA   = w_MEA * rho_sol / M_MEA_Kg    # mol/m^3

print(f"C_MEA = {C_MEA:.1f} mol/m^3")
# corresponding mole fractions (phi)
phiMEA = (w_MEA/M_MEA) / ((w_MEA/M_MEA) + ((1 - w_MEA)/M_H2O))

print(f"phi_MEA1 = {phiMEA:.4f}")


C_MEA = 4297.6 mol/m^3
phi_MEA1 = 0.0895


**Initialize Concentration**

In [7]:

c_init = 10.0
x0     = 0.0015                   # threshold location
h      = specialcf.mesh_size       # local element size (CF)
w      = 3.0*h + 1e-15             # ~3*h smoothing band (per element)

alpha  = 1.0 / (1.0 + exp(-2.0*(x - x0)/w))
initial_concentration = c_init * alpha

c_i.Set(initial_concentration)
c_i_old.Set(initial_concentration)

In [8]:
Draw(c_i,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

**Model Set Up**

In [9]:
alphaL_cf = 0.5 * (1 + phi)




In [10]:


alphaL = GridFunction(L) 
alphaL.Set(alphaL_cf) # projection/interpolation

In [11]:
Draw(Grad(alphaL),mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [12]:
tiny   = 1e-18

#Blended Diffusion Coeff
Dblend = (Dil * Dig) / (alphaL * Dig + (1 - alphaL) * Dil+tiny)

#Jump Correction
Gamma_i = -(1) * Dblend * (c_i * (1 - k_i) / (alphaL + k_i * (1 - alphaL))) * Grad(alphaL)

rxn_rate = exp(20.54 - ( 5612.91 / T ))

theta_term = (thetaL**2) / ((1 - 2 * thetaL)**2 + tiny)

#c_aq = exp(30.96 - 10584 / T - 7.187 * phiMEA * CO2_Loading) * theta_term / (KCO2 * Rgas * T) <---
c_aq = 10.5 #

In [13]:
# ---------------- TIME LOOP (consistent reassembly, CoefficientFunction dt) ----------------
t, Tfinal = 0.0, 0.03
t_idx = 0
beta = 5.0    # Nitsche penalty factor (tune 2–10)

# ----- time params -----
dt_val = 1e-4
dt = Parameter(dt_val)

# ----- spaces / unknowns -----
C = c_i.space
c, w = C.TnT()           # trial/test

# (optional) velocity:
vel = u_p.components[0]  # if you have NS solution

while t < Tfinal - 1e-12:
    # (optional) update phi, u from precomputed snapshots
    phi.vec.FV().NumPy()[:] = phi_snapshots[t_idx,:]
    vel.vec.FV().NumPy()[:]  = u_snapshots[t_idx,:]
    
    # --- recompute interface quantities ---
    alphaL_cf = 0.5 * (1 + phi)

    alphaL = GridFunction(L, name="alphaL")
    alphaL.Set(alphaL_cf)

    # --- drift / jump correction ---
    Gamma_i = -1 * Dblend * (c_i * (1 - k_i) / (alphaL + k_i * (1 - alphaL))) * Grad(alphaL)

    # --- prepare forms (fresh each iteration) ---
    a = BilinearForm(C, symmetric=False)
    f = LinearForm(C)

    # mass, advection, diffusion, drift
    a += (1.0/dt) * c * w * dx
    a += -(vel * grad(w)) * c * dx
    a += Dblend * grad(c) * grad(w) * dx
    a += c * (Gamma_i * grad(w)) * dx

    # SUPG stabilization
    h = specialcf.mesh_size
    vnorm = Norm(vel)
    tau = h / (2*(vnorm + 1e-10))
    a += tau * (vel * grad(w)) * (vel * grad(c)) * dx

    # --- time term RHS ---
    f += (1.0/dt) * c_i_old * w * dx

    # --- boundary integrals ---
    n = specialcf.normal(2)
    un = (vel * n)

    # Liquid inlet (weakly impose c=0)
    c_liq_in = 0.0
    a += IfPos(-un, -un * w * c, 0) * ds("Liq_Inlet")
    f += IfPos(-un, -un * w * c_liq_in, 0) * ds("Liq_Inlet")

    # Gas inlet (weakly impose c=c_in)
    c_in = 10.0
    a += IfPos(-un, -un * w * c, 0) * ds("Gas_Inlet")
    f += IfPos(-un, -un * w * c_in, 0) * ds("Gas_Inlet")

    # --- Nitsche penalty to tighten inflow condition ---
    a += beta * vnorm/h * (c * w) * ds("Gas_Inlet")
    f += beta * vnorm/h * (c_in * w) * ds("Gas_Inlet")

    # --- outlets (natural outflow) ---
    a += IfPos(un, un * w * c, 0) * ds("Liq_Outlet_a")
    a += IfPos(un, un * w * c, 0) * ds("Liq_Outlet_b")
    a += IfPos(un, un * w * c, 0) * ds("Gas_Outlet")

    # --- reaction term ---
    W_CO2 = rxn_rate * (c_i - c_aq) * C_MEA * alphaL
    a += -W_CO2 * w * dx

    # assemble and solve
    a.Assemble()
    f.Assemble()
    inv = a.mat.Inverse(C.FreeDofs())
    c_i.vec.data = inv * f.vec

    # --- clamp negative concentrations ---
    cvals = c_i.vec.FV().NumPy()
    cvals[cvals < 0] = 0.0
    cvals[cvals > c_in] = c_in
    # advance
    c_i_old.vec.data = c_i.vec
    t += dt_val
    t_idx += 1

    if t_idx % 30 == 0:
        
        Draw(c_i_old, mesh)
        Draw(phi,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [14]:
Draw(c_i_old,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [15]:
Draw(vel,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene